## baseline1版本，不参与建模的特征 [  'version', 'lan', 'sid’]

In [226]:
## 这个版本主要使用osv以及timestamp特征


## score1=88.128   加osv
## score2=88.10 加了timestamp  

In [227]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore') # 不会报warning错误

# 数据加载
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test1.csv')
test = test.iloc[:, 1:]
train = train.iloc[:, 1:]
train

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,46000.0,0.0,0.0,0.0,1,NaN,104,6.0,android,9,18,1438873,1.559893e+12,8,2135019403,0,2329670524,601
1,135939,893,0.0,0.0,0.0,0.0,1,NaN,19,6.0,android,8.1,0,1185582,1.559994e+12,4,2782306428,1,2864801071,1000
2,399254,821,0.0,760.0,0.0,360.0,1,NaN,559,0.0,android,8.1.0,0,1555716,1.559837e+12,0,1392806005,2,628911675,696
3,68983,1004,46000.0,2214.0,0.0,1080.0,0,NaN,129,2.0,android,8.1.0,0,1093419,1.560042e+12,0,3562553457,3,1283809327,753
4,288999,1076,46000.0,2280.0,0.0,1080.0,1,zh-CN,64,2.0,android,8.0.0,0,1400089,1.559867e+12,5,2364522023,4,1510695983,582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,392477,1028,46000.0,1920.0,3.0,1080.0,1,zh-CN,144,6.0,Android,7.1.2,25,1546078,1.559834e+12,7,861755946,79,140647032,373
499996,346134,1001,0.0,1424.0,0.0,720.0,0,NaN,29,2.0,android,8.1.0,0,1480612,1.559814e+12,3,1714444511,23,2745131047,525
499997,499635,761,46000.0,1280.0,0.0,720.0,0,NaN,54,6.0,android,6.0.1,9,1698442,1.559676e+12,0,3843262581,25,1326115882,810
499998,239786,917,46001.0,960.0,0.0,540.0,0,zh_CN,109,2.0,android,5.1.1,0,1331155,1.559840e+12,0,1984296118,225,1446741112,772


##### Object类型： lan, os, osv, version, fea_hash
##### 有缺失值的字段： lan, osv

In [228]:
features = train.columns.tolist()
features.remove('label')
print(features)

['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi', 'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'sid', 'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type']


In [229]:
for feature in features:
    print(feature, train[feature].nunique())

android_id 362258
apptype 89
carrier 5
dev_height 798
dev_ppi 92
dev_width 346
lan 21
media_id 284
ntt 8
os 2
osv 154
package 1950
sid 500000
timestamp 500000
version 22
fea_hash 402980
location 332
fea1_hash 4959
cus_type 58


In [230]:
train['fea_hash'].map(lambda x: len(str(x))).value_counts()
train['fea1_hash'].map(lambda x: len(str(x))).value_counts()
remove_list = ['os', 'lan', 'sid']
col = features
for i in remove_list:
    col.remove(i)


In [231]:
col

['android_id',
 'apptype',
 'carrier',
 'dev_height',
 'dev_ppi',
 'dev_width',
 'media_id',
 'ntt',
 'osv',
 'package',
 'timestamp',
 'version',
 'fea_hash',
 'location',
 'fea1_hash',
 'cus_type']

In [232]:
features = train[col]
features['fea_hash_len'] = features['fea_hash'].map(lambda x: len(str(x)))
features['fea1_hash_len'] = features['fea1_hash'].map(lambda x: len(str(x)))
features['fea_hash'] = features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features['fea1_hash'] = features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features = test[col]
test_features['fea_hash_len'] = test_features['fea_hash'].map(lambda x: len(str(x)))
test_features['fea1_hash_len'] = test_features['fea1_hash'].map(lambda x: len(str(x)))
test_features['fea_hash'] = test_features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features['fea1_hash'] = test_features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))

In [233]:
features['osv'].value_counts()#154个唯一值

8.1.0                                 151419
9                                      71332
6.0.1                                  31714
7.1.1                                  26503
9.0.0                                  24385
                                       ...  
2.2.2                                      1
11                                         1
3.2.0-FL2-20180726.9015                    1
2.0.1                                      1
f073b_changxiang_v01_b1b8_20180915         1
Name: osv, Length: 154, dtype: int64

In [234]:
test['osv'].value_counts()

8.1.0            45224
9                21460
6.0.1             9483
7.1.1             8098
9.0.0             7396
                 ...  
Android_4.1.1        1
4.4.6                1
GIONEE_YNGA          1
Android_4.2.1        1
5.0_21               1
Name: osv, Length: 130, dtype: int64

In [235]:
# 定义一个函数 
# 1.去掉'.'和' ' 和'_'
# 2.用众数填充 非数值 如果字符串中有，人能辨识的数值则用该数值填充
# 3. 如果是nan 则众数填充

# error ValueError: invalid literal for int() with base 10: '60十核20G_HD'
#ValueError: invalid literal for int() with base 10: 'Android512'

def app_osv(x):
    x=str(x).replace('Android_','').replace(' ','').replace('.','').replace('60十核20G_HD','').replace('Android','').replace('44W','')
    #
    if x=='nan' or x=='GIONEE_YNGA'or x=='f073b_changxiang_v01_b1b8_20180915' or x==''or x=='%E6%B1%9F%E7%81%B5OS+50':
        return 810
    elif x.count('-')>0:
        return  int(x.split('-')[0])
    else:
        res=int(x)
        if res<10:
            return res*100
        elif res<100:
            return res*10
    return res
            

In [236]:
features['osv'].apply(app_osv)#使用该每行进行运算

0         900
1         810
2         810
3         810
4         800
         ... 
499995    712
499996    810
499997    601
499998    511
499999    810
Name: osv, Length: 500000, dtype: int64

In [237]:
features['osv']=features['osv'].apply(app_osv)
test_features['osv']=test_features['osv'].apply(app_osv)

In [238]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   android_id     500000 non-null  int64  
 1   apptype        500000 non-null  int64  
 2   carrier        500000 non-null  float64
 3   dev_height     500000 non-null  float64
 4   dev_ppi        500000 non-null  float64
 5   dev_width      500000 non-null  float64
 6   media_id       500000 non-null  int64  
 7   ntt            500000 non-null  float64
 8   osv            500000 non-null  int64  
 9   package        500000 non-null  int64  
 10  timestamp      500000 non-null  float64
 11  version        500000 non-null  object 
 12  fea_hash       500000 non-null  int64  
 13  location       500000 non-null  int64  
 14  fea1_hash      500000 non-null  int64  
 15  cus_type       500000 non-null  int64  
 16  fea_hash_len   500000 non-null  int64  
 17  fea1_hash_len  500000 non-nul

In [240]:
# 加 timestamp 时间段   对于时间序列处理常用手段是改成年月日然后做切割
# 那么我们的思路就是把纯的数值转化成例如1970-1-1的日期
# >>> from datetime import datetime
# >>> t = 1429417200.0
# >>> print(datetime.fromtimestamp(t))
# 2015-04-19 12:20:00

In [241]:
# from datetime import datetime
# nt=datetime.now()
# print(nt)
# print(nt.timestamp())

In [242]:
from datetime import datetime
features['timestamp'].map(lambda x: len(str(x))).value_counts() # 15位 16位 17位 18位

18    392717
17     91963
16     13558
15      1762
Name: timestamp, dtype: int64

In [243]:
features['timestamp'] = features['timestamp'].apply(lambda x : datetime.fromtimestamp(x/1000))
# 分解时间
features['year'] = features['timestamp'].dt.year
features['month'] = features['timestamp'].dt.month
features['day'] = features['timestamp'].dt.day
features['weekday'] = features['timestamp'].dt.weekday
features['hour'] = features['timestamp'].dt.hour
features['minute'] = features['timestamp'].dt.minute

In [245]:
start_time = features['timestamp'].min()
features['timestamp_diff_start'] = features['timestamp']-start_time
features['timestamp_diff_start'] = features['timestamp_diff_start'].dt.days - features['timestamp_diff_start'].dt.seconds/3600/24

In [247]:
test_features['timestamp'] = test_features['timestamp'].apply(lambda x : datetime.fromtimestamp(x/1000))
# 分解时间
test_features['year'] = test_features['timestamp'].dt.year
test_features['month'] = test_features['timestamp'].dt.month
test_features['day'] = test_features['timestamp'].dt.day
test_features['weekday'] = test_features['timestamp'].dt.weekday
test_features['hour'] = test_features['timestamp'].dt.hour
test_features['minute'] = test_features['timestamp'].dt.minute
start_time = test_features['timestamp'].min()
test_features['timestamp_diff_start'] = test_features['timestamp']-start_time
test_features['timestamp_diff_start'] = test_features['timestamp_diff_start'].dt.days - test_features['timestamp_diff_start'].dt.seconds/3600/24

In [254]:
features

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,media_id,ntt,osv,package,...,cus_type,fea_hash_len,fea1_hash_len,year,month,day,weekday,hour,minute,timestamp_diff_start
0,316361,1199,46000.0,0.0,0.0,0.0,104,6.0,900,18,...,601,10,10,2019,6,7,4,15,32,3.352697
1,135939,893,0.0,0.0,0.0,0.0,19,6.0,810,0,...,1000,10,10,2019,6,8,5,19,40,4.180336
2,399254,821,0.0,760.0,0.0,360.0,559,0.0,810,0,...,696,10,9,2019,6,6,3,23,58,2.001204
3,68983,1004,46000.0,2214.0,0.0,1080.0,129,2.0,810,0,...,753,10,10,2019,6,9,6,8,59,5.625278
4,288999,1076,46000.0,2280.0,0.0,1080.0,64,2.0,800,0,...,582,10,10,2019,6,7,4,8,28,3.646991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,392477,1028,46000.0,1920.0,3.0,1080.0,144,6.0,712,25,...,373,9,9,2019,6,6,3,23,14,2.031748
499996,346134,1001,0.0,1424.0,0.0,720.0,29,2.0,810,0,...,525,10,10,2019,6,6,3,17,40,2.263287
499997,499635,761,46000.0,1280.0,0.0,720.0,54,6.0,601,9,...,810,10,10,2019,6,5,2,3,14,1.864630
499998,239786,917,46001.0,960.0,0.0,540.0,109,2.0,511,0,...,772,10,10,2019,6,7,4,0,59,3.958727


In [255]:
test_features

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,media_id,ntt,osv,package,...,cus_type,fea_hash_len,fea1_hash_len,year,month,day,weekday,hour,minute,timestamp_diff_start
0,317625,1181,46000.0,2196.0,2.0,1080.0,639,2.0,810,188,...,658,10,10,2019,6,7,4,9,43,3.594942
1,435108,944,46003.0,2280.0,3.0,1080.0,704,6.0,810,221,...,943,10,9,2019,6,5,2,20,53,1.129514
2,0,1106,46000.0,0.0,0.0,0.0,39,2.0,510,1562,...,411,9,10,2019,6,4,1,10,8,0.577535
3,451504,761,46000.0,1344.0,0.0,720.0,54,2.0,711,9,...,848,10,10,2019,6,5,2,1,3,1.956169
4,0,1001,46000.0,665.0,0.0,320.0,29,5.0,810,4,...,411,10,10,2019,6,5,2,8,16,1.655197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0,1001,46000.0,760.0,0.0,360.0,29,2.0,810,4,...,411,10,10,2019,6,8,5,9,21,4.610069
149996,0,1001,46000.0,780.0,0.0,360.0,29,2.0,900,4,...,411,8,10,2019,6,7,4,7,15,3.697292
149997,0,1001,46000.0,780.0,0.0,360.0,29,5.0,810,4,...,411,10,10,2019,6,9,6,8,48,5.632685
149998,500925,1052,46000.0,854.0,240.0,480.0,249,6.0,442,0,...,430,10,10,2019,6,5,2,6,38,1.723484


In [256]:
import lightgbm as lgb
model = lgb.LGBMClassifier() 
model.fit(features.drop(['timestamp', 'version'], axis=1), train['label'])
result = model.predict(test_features.drop(['timestamp', 'version'], axis=1))
res = pd.DataFrame(test['sid'])
res['label'] = result
res.to_csv('./baseline_timestamp.csv', index=False)
res

,sid,label
0,1440682,0
1,1606824,1
2,1774642,0
3,1742535,0
4,1689686,1
...,...,...
149995,1165373,1
149996,1444115,1
149997,1134378,1
149998,1700238,1
